In [2]:
%matplotlib nbagg

import sys 
import MySQLdb
import pandas as pd 
import matplotlib.pyplot as plt

from Trax.Utils.Conf.Configuration import Config
from Trax.Data.Projects.Connector import ProjectConnector
from Trax.Cloud.Services.Connector.Keys import DbUsers

Config.init(app_name=' ', config_file='~/theGarage/Trax/Apps/Services/KEngine/k-engine-prod.config')
Config.PROD 



connector = ProjectConnector('diageous', DbUsers.Docker)
static = '''SELECT * FROM  static.kpi_level_2'''
static_kpi = data = pd.read_sql_query(static, connector.db)
results = '''SELECT * FROM report.kpi_level_2_results'''
kpi_results = data = pd.read_sql_query(results, connector.db)
connector.disconnect_rds()

expected = pd.read_csv('expected.csv')



Log was not initialized - Unable to access vault while running in dev environment. (extra=None)
Log was not initialized - Unable to access vault while running in dev environment. (extra=None)
Log was not initialized - Unable to access vault while running in dev environment. (extra=None)
Log was not initialized - Deprecated usage of secret_keys file for username!!! (extra=None)
Log was not initialized - Deprecated usage of secret_keys file for password!!! (extra=None)
Log was not initialized - Credentials are updated from secret_keys configuration file! Application name:  . Mongo name: Smart (extra=None)
Log was not initialized - Open connection to SmartDev db. (extra=None)
Log was not initialized - Closed connection to SmartDev db. (extra=None)


No handlers could be found for logger "Trax"


### test - uncalculated KPI


In [36]:
kpi_results2 = static_kpi.merge(kpi_results, left_on='pk' ,right_on='kpi_level_2_fk', how='left')
kpi_results2 = kpi_results2.loc[kpi_results2['kpi_calculation_stage_fk'] == 3] # .groupby(by=['session_fk','client_name']).count()
kpi_results2[kpi_results2.isnull().result].client_name


1788930           Assortment on Trade
1788931     Assortment on Trade - SKU
1788932          Assortment off Trade
1788933    Assortment off Trade - SKU
Name: client_name, dtype: object

### test -invalid % results

In [ ]:
#load data
merged_kpi_results = static_kpi.merge(kpi_results, left_on='pk' ,right_on='kpi_level_2_fk', how='left')

# precent results should be between 0-1 
res = merged_kpi_results.loc[(merged_kpi_results['kpi_calculation_stage_fk'] == 3) & (merged_kpi_results['is_percent'] == 1 )]
res.result.plot()

print 
res[(res.result > 1) | (res.result < 0)][['session_fk','result']]

### test -results with value 0 

In [2]:
res = merged_kpi_results.loc[(merged_kpi_results['kpi_calculation_stage_fk'] == 3)]
print " % results with 0 "
print  len(res[res.result == 0].pk_x) / float(len(res.pk_x))*100
res[(res.result == 0)][['session_fk','client_name','result']]


NameError: name 'merged_kpi_results' is not defined

### histogram of results 

In [1]:
merged_kpi_results = static_kpi.merge(kpi_results, left_on='pk' ,right_on='kpi_level_2_fk', how='left')
res = merged_kpi_results.loc[(merged_kpi_results['kpi_calculation_stage_fk'] == 3)]
for i,row in static_kpi[static_kpi['kpi_calculation_stage_fk'] == 3].iterrows():   
    row['client_name']
    res = merged_kpi_results.loc[(merged_kpi_results['client_name'] == row['client_name'])]
    res[['client_name','result']].hist()
    plt.title(row['client_name'])

NameError: name 'static_kpi' is not defined

In [ ]:
### check if results are in expected range 
expected = pd.read_csv('expected.csv')

for i,row in expected.iterrows():    
    x = merged_kpi_results.loc[(merged_kpi_results['client_name'] == row['kpi_name']) & ((merged_kpi_results['result'] < row['min']) | (merged_kpi_results['result'] > row['max'])) ] 
    print "##### results not in expected range: (" + str(row['min']) + '-'+ str(row['max']) +' )'   + row['kpi_name']
    print x[['session_fk','result']]
    

In [11]:
## count uniqe results per kpi
merged_kpi_results = static_kpi.merge(kpi_results, left_on='pk' ,right_on='kpi_level_2_fk', how='left')
res = merged_kpi_results.loc[(merged_kpi_results['kpi_calculation_stage_fk'] == 3)]
for i,row in static_kpi[static_kpi['kpi_calculation_stage_fk'] == 3].iterrows(): 
   
    res = merged_kpi_results.loc[merged_kpi_results['client_name'] == row['client_name']]
    
    if res.empty: 
        pass 
    else: 
        res = res.groupby(['result'])
    
    if len(res.count().index) < 2:
        print '## there is only 2 result type for kpi ' + row['client_name'] 
        print res.count().get_values()


## there is only 2 result type for kpi National Score - Off Premise
[[193   0   0 193 193 193 193 193   0   0   0   0   0 193   0 193 193 193
  193 193 193 193   0 193 193 193 193 193 193 193 193   0   0 193 193   0
    0   0   0]]
## there is only 2 result type for kpi Segment Score - Off Premise
[[159   0   0 159 159 159 159 159   0   0   0   0   0 159   0 159 159 159
  159 159 159 159   0 159 159 159 159 159 159 159 159   0   0 159 159   0
    0   0   0]]
## there is only 2 result type for kpi MSRP - Total Score
[[176   0   0 176 176 176 176 176   0   0   0   0   0 176   0 176 176 176
  176 176 176 176   0 176 176 176 176 176 176 176 176   0 176 176 176   0
    0   0   0]]
## there is only 2 result type for kpi Assortment on Trade
[]
## there is only 2 result type for kpi Assortment on Trade - SKU
[]
## there is only 2 result type for kpi Assortment off Trade
[]
## there is only 2 result type for kpi Assortment off Trade - SKU
[]
## there is only 2 result type for kpi National Score